In [1]:
import sys

# add to path
sys.path.append('/workspaces/DeliveryTracker')

In [2]:
from UseCases.mietplan.session_handling import *

In [3]:
import requests
import json
from pathlib import Path

credentials = json.load(Path('mietplan_credentials.json').open())

session = requests.Session()
login(session, credentials['username'], credentials['password'])

In [4]:
folders = get_folders(session, MAIN_FOLDER_ID)

In [5]:
folders

[FolderMetadata(name=ETV, folder_id=adm9r0p4pdl8f37733143pqt0g6, has_subfolders=False),
 FolderMetadata(name=Vermögensübersicht, folder_id=ag975pffrdefar8bs64q0g74ijc, has_subfolders=False),
 FolderMetadata(name=Beschlussprotokolle, folder_id=aq00vmb30jvhql0fp04sdog4m3b, has_subfolders=False),
 FolderMetadata(name=Energieausweis, folder_id=a457sjdbrs5sdggo7ju5fui2jko, has_subfolders=False),
 FolderMetadata(name=Hausordnung + Tiefgaragenordnung, folder_id=avaisdk56no1t6u92r4b176h88j, has_subfolders=False),
 FolderMetadata(name=Notrufaushänge, folder_id=askc9sl6d61i2uk0juers4qpib6, has_subfolders=False),
 FolderMetadata(name=Objektfotos, folder_id=ar65b29d0qpn1dt85o6f3k3n55o, has_subfolders=True),
 FolderMetadata(name=Objektinformationen, folder_id=ah755ptqg9jqhhms3t2p0g8fmr4, has_subfolders=True),
 FolderMetadata(name=Rechtsfälle, folder_id=a46pmuu219oq500gmo284i5fv57, has_subfolders=False),
 FolderMetadata(name=Teilungserklärung + Gemeinschaftsordnung, folder_id=a936qi7fmo6bvsh68rseqrm

In [6]:
files = get_files(session, folders[0].folder_id)
files

[FileMetadata(creation_date=2025-02-12 00:00:00, download_path=/download/ahucgga1rbps3ukufhc3mnlf9ei/Information zum Termin ETV 2025.pdf, filename=Information zum Termin ETV 2025.pdf)]

In [9]:
from dataclasses import dataclass
from datetime import datetime


@dataclass
class File:
    creation_date: datetime
    name: str
    url: str

@dataclass
class Folder:
    id: str
    path: list[str]
    files: list[File]

In [13]:
def walk_from_top_folder(session, top_folder_id: str):
    folders_to_check = [Folder(folder.folder_id, [folder.name], []) for folder in get_folders(session, top_folder_id)]

    while folders_to_check:
        folder = folders_to_check.pop()

        files = get_files(session, folder.id)
        for file in files:
            folder.files.append(File(file.creation_date, file.filename, file.download_path))

        yield folder

        # files = get_files(session, folder.id)
        sub_folders = get_folders(session, folder.id)
        for sub_folder in sub_folders:
            folders_to_check.append(Folder(sub_folder.folder_id, folder.path + [sub_folder.name], []))

In [ ]:
for folder in walk_from_top_folder(session, MAIN_FOLDER_ID):
    print(folder.path)
    for file in folder.files:
        print(file.name)